In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
##MNIST图片28*28
n_inputs=28
max_time=28

lstm_size=100 ###隐层单元
n_classes=10  ###10个分类

batch_size=128
n_batch=mnist.train.num_examples // batch_size


with tf.name_scope('input'):
##定义placeholder
    x=tf.placeholder(tf.float32,[None,784], name='x-input')
    y=tf.placeholder(tf.float32,[None,10], name='y-input')
    
    weights=tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1))
    biases=tf.Variable(tf.constant(0.1,shape=[n_classes]))
    
    
with tf.name_scope('layer'):
    def RNN(x,weights,biases):
        inputs=tf.reshape(x,[-1,max_time,n_inputs])
        lstm_cell=tf.contrib.rnn.BasicLSTMCell(lstm_size)
        ##final_state[0] cell state
        ##final_state[1] hidden state
        outputs,final_state=tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
        results = tf.nn.softmax(tf.matmul(final_state[1],weights)+biases)
        return results
    prediction=RNN(x,weights,biases)


with tf.name_scope('loss'):
##二次代价函数
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    tf.summary.scalar('loss', loss)
    
with tf.name_scope('train_step'):
    train_step=tf.train.AdamOptimizer(1e-3).minimize(loss)

init = tf.global_variables_initializer()

with tf.name_scope('correct_prediction'):
    correct_prediction=tf.equal(tf.argmax(y,1), tf.argmax(prediction,1))

with tf.name_scope('accuarcy'):
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

merged=tf.summary.merge_all()

saver=tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    writer=tf.summary.FileWriter('logs/',sess.graph)
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys= mnist.train.next_batch(batch_size)
            summary, _ = sess.run([merged,train_step], feed_dict={x:batch_xs, y:batch_ys})
            
        writer.add_summary(summary, epoch)    
        acc=sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print("iter" + str(epoch)+", testing accuracy:" + str(acc))
    
    saver.save(sess,'net/MNIST_RNN.ckpt')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

iter0, testing accuracy:0.8826
iter1, testing accuracy:0.9252
iter2, testing accuracy:0.9471
iter3, testing accuracy:0.9531
iter4, testing accuracy:0.9547
iter5, testing accuracy:0.9585
iter6, testing accuracy:0.9697
iter7, testing accuracy:0.9656
iter8, testing accuracy:0.9702
iter9, testing accuracy:0.9724
iter10, testing accuracy:0.971
iter11, testing accuracy:0.9764
iter12, testing accuracy:0.9726
iter13, testing accuracy:0.9792
iter14, testing accuracy:0.9784
iter15, testing accuracy:0.9801
iter16, tes

In [3]:
with tf.Session() as sess:
    sess.run(init)
    acc_notrain=sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
    print(acc_notrain)
    saver.restore(sess,'net/MNIST_RNN.ckpt')
      
    acc=sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
    print(acc)
    


0.0532
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from net/MNIST_RNN.ckpt
0.9816
